### Step 1. Download InfoRE dataset

In [ ]:
%%capture
%%bash
mkdir -p /content/data
cd /content/data
wget https://huggingface.co/datasets/ntt123/infore/resolve/main/infore_16k.zip
# unzip -P BroughtToYouByInfoRe 25hours.zip
unzip infore_16k.zip

### Step 2. Normalize audio clip

In [ ]:
%%capture
!sudo apt install -y sox
!pip install soundfile librosa
!pip install onnxruntime==1.11.1

In [ ]:
!mkdir -p /content/infore_16k
from pathlib import Path
import os
from tqdm.cli import tqdm

wavs = sorted(Path("/content/data/InfoRe").glob("*.wav"))
for path in tqdm(wavs):
    out = Path("/content/infore_16k") / path.name
    cmd = f"sox {path} -c 1 -e signed-integer -b 16 -r 16k --norm=-3 {out}"
    os.system(cmd)

### Step 3. Denoise using DNS-Challenge's baseline

In [ ]:
!git clone https://github.com/microsoft/DNS-Challenge
%cd DNS-Challenge/NSNet2-baseline/
!git checkout -f 8b87a33b2892f147b5c7ad39ea978453730db269
!python run_nsnet2.py -i /content/infore_16k/ -o /content/infore_16k_denoised -m ./nsnet2-20ms-baseline.onnx

### Step 4. Zip the denoised dataset

In [ ]:
%cd /content
!cp /content/data/InfoRe/*.txt ./infore_16k_denoised
!cd ./infore_16k_denoised; zip -r ../infore_16k_denoised.zip .